# 2. Cumpute normal NDVI Curve in EarthEngine

## 2.1 Setup

In [0]:
%%capture
!pip install earthengine-api

In [0]:
import os
import ee
from google.colab import drive

In [3]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
os.chdir('/content/drive/My Drive')
print('Working directory set to : %s' % os.getcwd())

Working directory set to : /content/drive/My Drive


In [0]:
base_path = "/content/drive/My Drive/DroughtMonitor"

data_path = os.path.join(base_path, "Data")
raster_data_path = os.path.join(data_path, "Raster")
normal_NDVI_path = os.path.join(raster_data_path, "Normal_NDVIs")
vector_data_path = os.path.join(data_path, "Vector")

utils_path = os.path.join(base_path, "Utils")

In [6]:
!earthengine authenticate

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/rgFaJ5GUuyZztmM7oRM5SnoFGBmj4-an6ivOzqen1bi

In [7]:
try:
  ee.Initialize()
except Exception as e:
  print('EarthEngine Initialization failed: %s' % e)
else:
  print('EarthEngine initialized successfully.')

EarthEngine initialized successfully.


## 2.2 Data Selection

In [0]:
de_boundaries = ee.FeatureCollection("users/grothsandro/german_boundaries")

In [0]:
def clip_image(image):
  return image.clip(de_boundaries)

In [10]:
MODISTerraNDVI = ee.ImageCollection('MODIS/006/MOD13Q1')\
                    .filterBounds(de_boundaries)\
                    .filterDate('2000-01-01', '2015-12-31')\
                    .map(clip_image)
MODISAquaNDVI = ee.ImageCollection('MODIS/006/MYD13Q1')\
                    .filterBounds(de_boundaries)\
                    .filterDate('2000-01-01', '2015-12-31')\
                    .map(clip_image)
MODISNDVI = MODISTerraNDVI.merge(MODISAquaNDVI)\
                          .sort('system:time_start')

print('Complete MODIS VI containing %s images.' % MODISNDVI.size().getInfo())

Complete MODIS VI containing 676 images.


## 2.3 Cloud Masking

In [0]:
def cloud_mask_MOD13A2(image):
  qa = image.select('SummaryQA')
  clouds_bit_mask = 1 << 1
  mask = qa.bitwiseAnd(clouds_bit_mask).eq(0)
  
  return image.updateMask(mask)

In [0]:
MODISNDVI = MODISNDVI.map(cloud_mask_MOD13A2)

## 2.4 NDVI Aggregation

In [0]:
def add_rescaled_NDVI(image):
  rescaled_NDVI = image.select('NDVI')\
                        .multiply(0.0001)\
                        .rename('NDVI_rescaled')
  return image.addBands(rescaled_NDVI)

In [0]:
def set_image_DOY(image):
  return image.set('doy', ee.Date(image.get('system:time_start')).getRelative('day', 'year'))

In [0]:
MODISNDVI = MODISNDVI.map(add_rescaled_NDVI)

In [0]:
withDOY = MODISNDVI.map(set_image_DOY)

In [0]:
distinctDOY = withDOY.filterDate('2013-01-01', '2014-01-01')

In [0]:
def calc_means(image):
  doy = image.get('doy')
  doy_aqs = withDOY.filter(ee.Filter.eq('doy', doy))\
                    .select('NDVI_rescaled')
  mean = doy_aqs.reduce(ee.Reducer.mean())\
                .rename(['NDVI'])\
                .set('doy', doy)\
                .set('system:id', doy)
  return mean.addBands(doy_aqs.reduce(ee.Reducer.stdDev()).rename(['STD']))

In [0]:
Means = distinctDOY.map(calc_means)

## 2.5 Export to Drive

In [0]:
# TODO